In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import os
import numpy as np
import random
import time
import sys
import matplotlib.pyplot as plt

from PIL import Image
from torchvision import datasets , transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

import multiprocessing

import seaborn as sns
import pandas as pd

Random seed 고정

In [ ]:
seed = 40
deterministic = True

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
if deterministic:
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

google drive mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


데이터셋 가져오기

In [ ]:
# 데이터를 저장할 폴더 생성
data_root = '/content/drive/MyDrive/imagesnet_data'
os.makedirs(data_root, exist_ok=True)

In [ ]:
# 4. uploader.py 실행
!python /content/drive/MyDrive/ImageNet-Datasets-Downloader-master/downloader.py \
    -data_root /content/drive/MyDrive/imagesnet_data \
    -number_of_classes 5 \
    -images_per_class 5

Picked the following clases:
['small ship', 'ancient', 'turkey cock', 'prancer', 'header']
Scraping images for class "small ship"
Multiprocessing workers: 8
Scraping images for class "ancient"
Multiprocessing workers: 8
Scraping images for class "turkey cock"
Multiprocessing workers: 8
Scraping images for class "prancer"
Multiprocessing workers: 8
Scraping images for class "header"
Multiprocessing workers: 8


손상된 이미지 자동 삭제

In [ ]:
def verify_and_remove(file_path):
    try:
        with Image.open(file_path) as img:
            img.verify()
    except (IOError, OSError, Image.DecompressionBombError) as e:
        print(f"Deleting corrupted image: {file_path}, Error: {e}")
        os.remove(file_path)

def process_directory(root_dir):
    tasks = []
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            file_path = os.path.join(subdir, file)
            tasks.append(file_path)

    # 멀티프로세싱 Pool 활용
    with multiprocessing.Pool() as pool:
        pool.map(verify_and_remove, tasks)

root_dir = "/content/drive/MyDrive/imagesnet_data/imagenet_images"
process_directory(root_dir)

Deleting corrupted image: /content/drive/MyDrive/imagesnet_data/imagenet_images/fly orchid/2596668041_4d48d0c515.jpg, Error: cannot identify image file '/content/drive/MyDrive/imagesnet_data/imagenet_images/fly orchid/2596668041_4d48d0c515.jpg'
Deleting corrupted image: /content/drive/MyDrive/imagesnet_data/imagenet_images/fly orchid/2596668085_22bbfcb6b0.jpg, Error: cannot identify image file '/content/drive/MyDrive/imagesnet_data/imagenet_images/fly orchid/2596668085_22bbfcb6b0.jpg'


두개의 관점(논문에 나온 이미지처럼)을 얻을 것이므로 단일 이미지에 대해 augmentation을 두번씩 진행한다

ResNet50이 backbone으로 사용

In [ ]:
!pip install timm
import timm

In [ ]:
resnet_1 = timm.create_model('resnet50', pretrained=True)
resnet_2 = timm.create_model('resnet50', pretrained=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

최종 레이어(backbone(=resnet50)의 encoder로 사용된다) 제거

In [ ]:
del resnet_1.fc
del resnet_2.fc

BN이 포함된 three layer MLP를 사용한다

In [ ]:
resnet_1.fc = nn.Sequential(
    nn.Linear(2048, 2048),
    nn.BatchNorm1d(2048),
    nn.ReLU(),
    nn.Linear(2048,2048),
    nn.BatchNorm1d(2048),
    nn.ReLU(),
    nn.Linear(2048,2048),
    nn.BatchNorm1d(2048),
    nn.ReLU()
)

resnet_2.fc = nn.Sequential(
    nn.Linear(2048, 2048),
    nn.BatchNorm1d(2048),
    nn.ReLU(),
    nn.Linear(2048,2048),
    nn.BatchNorm1d(2048),
    nn.ReLU(),
    nn.Linear(2048,2048),
    nn.BatchNorm1d(2048),
    nn.ReLU()
)

model parameter freeze , final block non freeze

In [ ]:
for param in resnet_1.parameters():
  param.requires_grad = False
for param in resnet_1.fc.parameters():
  param.requires_grad = True

for param in resnet_2.parameters():
  param.requires_grad = False
for param in resnet_2.fc.parameters():
  param.requires_grad = True

resnet50 데이터 정규화 리스트


In [ ]:
mean = [0.485,0.456,0.406]
std = [0.229,0.224,0.225]

이미지 전처리

In [ ]:
# 이미지 변환 파이프라인 정의
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 먼저 크기 조정
    transforms.RandomResizedCrop(224),  # 그 후 임의의 크기로 자르기
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),  # 색상 변화
    transforms.RandomGrayscale(p=0.2),  # 20% 확률로 흑백 변환
    transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 2.0)),  # 가우시안 블러
    transforms.RandomHorizontalFlip(p=0.5),  # 50% 확률로 좌우 뒤집기
    transforms.ToTensor(),  # 마지막에 텐서 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
])

In [ ]:
aug1 = transforms.Compose([
    transforms.Resize((672, 224)),  # 크기 조정
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),  # 색상 변화
    transforms.RandomGrayscale(p=0.2),  # 20% 확률로 흑백 변환
    transforms.RandomResizedCrop(224),  # 임의 크기로 자르기
    transforms.RandomHorizontalFlip(p=0.5),  # 좌우 뒤집기
    transforms.ToTensor(),  # 마지막에 텐서로 변환
    transforms.Normalize(mean=mean, std=std)  # 정규화
])

In [ ]:
aug2 = transforms.Compose([
    transforms.Resize((672, 224)),  # 크기 조정
    transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 5)),  # 가우시안 블러
    transforms.RandomResizedCrop(224),  # 임의 크기 자르기
    transforms.ToTensor(),  # 마지막에 텐서로 변환
    transforms.Normalize(mean=mean, std=std)  # 정규화
])

훈련 loop

optimizer : LARS

---


batch size : 4096

---


momentum : 0.9

---


weight_decay : None

---


learning rate : base 0.03 , cosine decay learning rate scheduler

---


epoch : 100

optimizer 설치

In [ ]:
!pip install torchlars
from torchlars import LARS

  Preparing metadata (setup.py) ... done
  Created wheel for torchlars: filename=torchlars-0.1.2-cp310-cp310-linux_x86_64.whl size=2782515 sha256=32850212114916f8c12772cf3ac7ca87c63ccec1f4e3a6a3f0c691a34249bd62
  Stored in directory: /root/.cache/pip/wheels/b4/91/c8/c631927a6e259206b572e7461e6d049b4379d27523c1065e30
Successfully built torchlars


data_loading

In [ ]:
dataset = datasets.ImageFolder(root=data_root, transform=transform)
dataloader = DataLoader(dataset, batch_size=1024, shuffle=True)

proposed_dataset = datasets.ImageFolder(root=data_root, transform=False)
proposed_dataloader = DataLoader(proposed_dataset, batch_size=1024, shuffle=True)

Linear evaluation

In [ ]:
base_optimizer_linear = torch.optim.Adam(resnet_1.fc.parameters(), lr=0.03)
optimizer_linear = LARS(optimizer=base_optimizer_linear, trust_coef=0.001)
scheduler_linear = CosineAnnealingLR(optimizer_linear, T_max=100, eta_min=0.0003)

In [ ]:
cnt = 0
loss_history = []
test_loss_history = []
ce_loss = nn.CrossEntropyLoss()
resnet_1 = resnet_1.to(device)

resnet_1.train()
for epoch in range(100):
  for images, labels in dataloader:

    images = images.to(device)
    labels = labels.to(device)

    images = images.reshape(-1,3,224,224)
    labels = labels.long()

    output = resnet_1(images)
    loss = ce_loss(output, labels)

    # Backward and optimize
    optimizer_linear.zero_grad()
    loss.backward()
    optimizer_linear.step()

    # loss history append
    loss_history.append(loss.item())

  scheduler_linear.step()
  print("epoch : {} , loss : {}".format(epoch, loss))

# 손실값 시각화
epochs = range(1, len(loss_history) + 1)

plt.plot(epochs, loss_history, label='Train Loss')
#plt.plot(epochs, val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()
plt.show()

### 분포 확인
data = {
    'epoch': epochs,
    'loss': loss_history,
    #'type': ['Train'] * 6 + ['Validation'] * 6
    'type': ['Train'] * 6
}
df = pd.DataFrame(data)

sns.lineplot(x='epoch', y='loss', hue='type', data=df)
plt.title('Train vs Validation Loss')
plt.show()

/usr/local/lib/python3.10/dist-packages/torchlars/lars.py:140: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha = 1) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1581.)
  p.grad.add_(weight_decay, p.data)


epoch : 0 , loss : 7.285643577575684
epoch : 1 , loss : 6.983994483947754
epoch : 2 , loss : 6.802204132080078
epoch : 3 , loss : 6.663614273071289


KeyboardInterrupt: 

Matrix-SSL

In [ ]:
base_optimizer_propose = torch.optim.Adam(resnet_2.fc.parameters(), lr=0.03)
optimizer_propose = LARS(optimizer=base_optimizer_propose, trust_coef=0.001)
scheduler_propose = CosineAnnealingLR(optimizer_propose, T_max=100, eta_min=0.0003)

align_loss = -tr(c(z_1,z_2)) + gamma*MCE(c(z_1,z_1),c(z_2,z_2))

---


uniform_loss = MCE((1/d)*identity_d , c(z_1,z_2))

In [ ]:
cnt = 0
loss_history = []
test_loss_history = []
ce_loss = nn.CrossEntropyLoss()
resnet_2 = resnet_2.to(device)

resnet_2.train()
for epoch in range(100):
  for images, labels in proposed_dataloader:

    images = images.to(device)
    labels = labels.to(device)
    labels = labels.long()

    z_1 = aug1(images).reshape(-1,3,224,224)
    z_2 = aug2(images).reshape(-1,3,224,224)

    output_1 = resnet_2(z_1)
    output_2 = resnet_2(z_2)

    # Matrix의 대각합
    # MCE(p,q) = tr(-plog(q) + q)

    #loss 함수 정의하기
    align_loss = ...
    uniform_loss = ...

    loss = align_loss + uniform_loss

    # Backward and optimize
    optimizer_linear.zero_grad()
    loss.backward()
    optimizer_linear.step()

    # loss history append
    loss_history.append(loss.item())

  scheduler_linear.step()
  print("epoch : {} , loss : {}".format(epoch, loss))

test